In [ ]:
#| hide
import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor

# Syncing through nbdev

> Showcasing the syncing capability

In [ ]:
%load_ext nbmodular.core.cell2func

## simple case: each function is written in a single cell

This is markdown

In [ ]:
%%function first_function
a = 3 
print ('a', a)

a 3
Stored the following local variables in the first_function current_values dictionary: ['a']


In [ ]:
# normal code cell
print ('outside', a)

outside 3


In [ ]:
%%function second_function
b = 4
c = a+b
print (a, b, c)

3 4 7
Stored the following local variables in the second_function current_values dictionary: ['b', 'c']


In [ ]:
cell_processor = %cell_processor

In [ ]:
[function['name'] for function in cell_processor.cell_nodes]

['first_function', 'second_function']

In [ ]:
print ('last cell:', a, b, c)

last cell: 3 4 7


In [ ]:
class FunctionNBProcessor(Processor):
    def __init__ (self, function_cells):
        self.function_cells = function_cells
        self.function_idx = 0
    
    def cell(self, cell):
        cell_lines = cell.source.splitlines()
        if len(cell_lines) > 0 and '%%function' in cell_lines[0]:
            if self.function_idx < len(self.function_cells):
                print ('\n**********************************************************************')
                print ('#| export')
                print (self.function_cells[self.function_idx]['code'])
                self.function_idx += 1
                print ('**********************************************************************\n')
        else:
            print (cell.source)

            
function_processor = FunctionNBProcessor (cell_processor.cell_nodes)
NBProcessor ('./nbdev_sync.ipynb', function_processor).process()

import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor
# Syncing through nbdev

> Showcasing the syncing capability
%load_ext nbmodular.core.cell2func
## simple case: each function is written in a single cell
This is markdown

**********************************************************************
#| export
def first_function():
    a = 3 
    print ('a', a)
    return a

**********************************************************************

# normal code cell
print ('outside', a)

**********************************************************************
#| export
def second_function():
    b = 4
    c = a+b
    print (a, b, c)
    return b,c

**********************************************************************

cell_processor = %cell_processor
[function['name'] for function in cell_processor.cell_nodes]
print ('last cell:', a, b, c)
class FunctionNBProcessor(Processor):
    def __init__ (self, 

## second case: functions are split into different cells

In [ ]:
%%function myf
print ('hello')
a = 3

hello
Stored the following local variables in the myf current_values dictionary: ['a']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    return a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\n"

In [ ]:
%%function myf --merge
b = 4

Stored the following local variables in the myf current_values dictionary: ['b']
hello
Stored the following local variables in the myf current_values dictionary: ['a', 'b']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    b = 4
    return b,a



In [ ]:
%%function myf --merge
c = 5
d = a+b+c

Stored the following local variables in the myf current_values dictionary: ['c', 'd']
hello
Stored the following local variables in the myf current_values dictionary: ['a', 'b', 'c', 'd']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    b = 4
    c = 5
    d = a+b+c
    return b,d,c,a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\nb = 4\nc = 5\nd = a+b+c\n"

In [ ]:
function_processor = FunctionNBProcessor (cell_processor.cell_nodes)
NBProcessor ('./nbdev_sync.ipynb', function_processor).process()

import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor
# Syncing through nbdev

> Showcasing the syncing capability
%load_ext nbmodular.core.cell2func
## simple case: each function is written in a single cell
This is markdown

**********************************************************************
#| export
def first_function():
    a = 3 
    print ('a', a)
    return a

**********************************************************************

# normal code cell
print ('outside', a)

**********************************************************************
#| export
def second_function():
    b = 4
    c = a+b
    print (a, b, c)
    return b,c

**********************************************************************

cell_processor = %cell_processor
[function['name'] for function in cell_processor.cell_nodes]
print ('last cell:', a, b, c)
class FunctionNBProcessor(Processor):
    def __init__ (self, 

### (TODO) Use only the last cell associated with this function, e.g., by replacing previous cells.

## third case: have defined functions in cells

In [ ]:
x=4

In [ ]:
%%function 
def my_defined_function (x, a=3):
    print (x)
    print (a)

4
3
Stored the following local variables in the my_defined_function current_values dictionary: []


In [ ]:
function_processor = FunctionNBProcessor (cell_processor.cell_nodes)
NBProcessor ('./nbdev_sync.ipynb', function_processor).process()

import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor
# Syncing through nbdev

> Showcasing the syncing capability
%load_ext nbmodular.core.cell2func
## simple case: each function is written in a single cell
This is markdown

**********************************************************************
#| export
def first_function():
    a = 3 
    print ('a', a)
    return a

**********************************************************************

# normal code cell
print ('outside', a)

**********************************************************************
#| export
def second_function():
    b = 4
    c = a+b
    print (a, b, c)
    return b,c

**********************************************************************

cell_processor = %cell_processor
[function['name'] for function in cell_processor.cell_nodes]
print ('last cell:', a, b, c)
class FunctionNBProcessor(Processor):
    def __init__ (self, 

### (TODO) Debug why the code of last function doesn't appear at all

## end